In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,773 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,351 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal 

In [3]:

# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
df.createOrReplaceTempView("home_sales")

In [7]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [8]:
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import col

df.withColumn("id", col("id").cast("int"))
df.withColumn("date_built", col("date_built").cast("int"))
df.withColumn("price", col("price").cast("double"))
df.withColumn("bedrooms", col("bedrooms").cast("int"))
df.withColumn("bathrooms", col("bathrooms").cast("int"))

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: int, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
sql="""select YEAR(date) as year, round(avg(price),2) as average_price
from home_sales where bedrooms=4 group by year order by year desc """
spark.sql(sql).show()



+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
df3bedbath=spark.sql("""select YEAR(date) as year, round(avg(price),2) as average_price
from home_sales where bedrooms=3 and bathrooms =3 group by year order by year desc""")

df3bedbath.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [11]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

df3bed3bath2f=spark.sql("""select YEAR(date) as year, round(avg(price),2) as average_price
from home_sales where bedrooms=3 and bathrooms =3 and floors=2 and sqft_living>=2000 group by year order by year desc""")
df3bed3bath2f.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    290242.99|
|2021|    296330.96|
|2020|    292289.09|
|2019|    289859.14|
+----+-------------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view=spark.sql("""select YEAR(date) as year, round(avg(price),2) as average_price, view
from home_sales where price>=350000 group by year, view order by view desc""")
view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+----+
|year|average_price|view|
+----+-------------+----+
|2020|   1059749.25|  99|
|2021|    926397.67|  99|
|2019|   1134806.56|  99|
|2022|    1102152.0|  99|
|2020|    1071385.0|  98|
|2021|     869964.4|  98|
|2019|    1223523.5|  98|
|2022|   1005757.83|  98|
|2022|   1366771.67|  97|
|2021|   1203860.56|  97|
|2020|    997334.86|  97|
|2019|    1070960.0|  97|
|2022|     646740.0|  96|
|2021|   1046958.67|  96|
|2019|    956291.94|  96|
|2020|    1147122.5|  96|
|2021|   1013915.08|  95|
|2020|    1054388.5|  95|
|2019|    1133564.6|  95|
|2022|    832949.33|  95|
+----+-------------+----+
only showing top 20 rows

--- 1.2992448806762695 seconds ---


In [26]:
start_time = time.time()

view=spark.sql(""" select round(avg(price),2) as average_price, view
from home_sales where price>=350000 group by view order by average_price""")
view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|    393973.32|  33|
|    394964.36|  37|
|    395655.38|   6|
|     396964.5|  42|
|    397389.25|   2|
|    397771.65|  29|
|     397862.0|  30|
|    398418.37|  38|
|     398447.5|  47|
|    398474.49|  17|
|    398570.03|  14|
|    398592.71|   8|
|     398867.6|   3|
|    398917.98|  13|
|    398953.17|  19|
|    399234.22|  39|
|    399332.91|  18|
|    399367.72|  32|
|    399522.81|  20|
|    399537.66|  27|
+-------------+----+
only showing top 20 rows

--- 1.0574212074279785 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

view2=spark.sql("""select YEAR(date) as year, round(avg(price),2) as average_price, view
from home_sales where price>=350000 group by year, view order by view desc""")
view2.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+----+
|year|average_price|view|
+----+-------------+----+
|2020|   1059749.25|  99|
|2021|    926397.67|  99|
|2019|   1134806.56|  99|
|2022|    1102152.0|  99|
|2020|    1071385.0|  98|
|2021|     869964.4|  98|
|2019|    1223523.5|  98|
|2022|   1005757.83|  98|
|2022|   1366771.67|  97|
|2021|   1203860.56|  97|
|2020|    997334.86|  97|
|2019|    1070960.0|  97|
|2022|     646740.0|  96|
|2021|   1046958.67|  96|
|2019|    956291.94|  96|
|2020|    1147122.5|  96|
|2021|   1013915.08|  95|
|2020|    1054388.5|  95|
|2019|    1133564.6|  95|
|2022|    832949.33|  95|
+----+-------------+----+
only showing top 20 rows

--- 0.8008065223693848 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("home_sales_parquet")

In [17]:
# 11. Read the parquet formatted data.
dfparquet=spark.read.parquet("home_sales_parquet")
dfparquet.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [18]:
# 12. Create a temporary table for the parquet data.
dfparquet.createOrReplaceTempView("home_sales_parquet")

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

view2=spark.sql("""select YEAR(date) as year, round(avg(price),2) as average_price, view
from home_sales where price>=350000 group by year, view order by view desc""")
view2.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+----+
|year|average_price|view|
+----+-------------+----+
|2020|   1059749.25|  99|
|2021|    926397.67|  99|
|2019|   1134806.56|  99|
|2022|    1102152.0|  99|
|2020|    1071385.0|  98|
|2021|     869964.4|  98|
|2019|    1223523.5|  98|
|2022|   1005757.83|  98|
|2022|   1366771.67|  97|
|2021|   1203860.56|  97|
|2020|    997334.86|  97|
|2019|    1070960.0|  97|
|2022|     646740.0|  96|
|2021|   1046958.67|  96|
|2019|    956291.94|  96|
|2020|    1147122.5|  96|
|2021|   1013915.08|  95|
|2020|    1054388.5|  95|
|2019|    1133564.6|  95|
|2022|    832949.33|  95|
+----+-------------+----+
only showing top 20 rows

--- 0.8360209465026855 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False